# Demo of importing dataset from csv
Aside from a basic-import, we do some sanity-checks too.

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv(filepath_or_buffer="data/sub02/50N_60deg_standing_01.csv", sep=";", header=0, index_col=0)

In [6]:
df

,step,handrail
trial,,
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
